## 250603 테스트

* 정의된 함수 옮기기
* 정의된 클래스 옮기기
* hcx-005 모델 테스트

- time_filter 함수에서 문제 발생
- 날짜 관련 수식 적용이 OpenAI와 다르게 움직이므로, 해당 부분 디버깅 필요

In [1]:
import os
import re
import json
import jsonlines
from langchain.schema import Document
from langchain_experimental.text_splitter import SemanticChunker
from langchain_naver.embeddings import ClovaXEmbeddings
from langchain_milvus.vectorstores import Milvus
from uuid import uuid4
from langchain_naver.chat_models import ChatClovaX
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

import pandas as pd
import pytz

from datasets import Dataset
from datetime import timedelta
from operator import itemgetter
from langchain_teddynote.retrievers import KiwiBM25Retriever
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import (
  AttributeInfo,
  StructuredQueryOutputParser,
  get_query_constructor_prompt
)
from langchain_teddynote.evaluator import GroundednessChecker
from langchain.retrievers.self_query.milvus import MilvusTranslator
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
import warnings
from langchain_core.runnables import chain

warnings.filterwarnings('ignore')

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
embeddings = ClovaXEmbeddings(
    model='bge-m3'
)

In [16]:
adjust_time_filter_to_week('2025년 1월 25일')

AttributeError: 'str' object has no attribute 'start_date'

In [4]:
def adjust_time_filter_to_week(time_filter):
    """
    특정 날짜(YYYY-MM-DD)가 주어진 경우, 해당 날짜를 포함하는 주(월~일)의
    첫 번째 날(월요일)과 마지막 날(일요일)로 변환하는 함수.

    :param time_filter: dict, {"start_date": datetime, "end_date": datetime}
    :return: dict, {"start_date": datetime, "end_date": datetime}
    """
    # Extract start_date and end_date from time_filter
    start_date = time_filter.start_date
    end_date = time_filter.end_date

    # Handle the case where start_date or end_date is None
    if start_date is None or end_date is None:
        if start_date is not None and end_date is None:
            start_of_week = start_date - timedelta(days=start_date.weekday())  # 월요일 찾기
            end_of_week = start_of_week + timedelta(days=6)  # 해당 주 일요일 찾기

            return {
                "start_date": start_of_week.replace(hour=0, minute=0, second=0),
                "end_date": end_of_week.replace(hour=23, minute=59, second=59)
            }
        elif end_date is not None and start_date is None:
            start_of_week = end_date - timedelta(days=end_date.weekday())  # 월요일 찾기
            end_of_week = start_of_week + timedelta(days=6)  # 해당 주 일요일 찾기

            return {
                "start_date": start_of_week.replace(hour=0, minute=0, second=0),
                "end_date": end_of_week.replace(hour=23, minute=59, second=59)
            }
        else:
            return None  # or return the time_filter as is if you prefer

    # 날짜가 동일한 경우, 주의 첫 번째 날(월요일)과 마지막 날(일요일)로 변경
    if start_date.year == end_date.year and start_date.month==end_date.month and start_date.day==end_date.day:
        start_of_week = start_date - timedelta(days=start_date.weekday())  # 월요일 찾기
        end_of_week = start_of_week + timedelta(days=6)  # 해당 주 일요일 찾기

        return {
            "start_date": start_of_week.replace(hour=0, minute=0, second=0),
            "end_date": end_of_week.replace(hour=23, minute=59, second=59)
        }

    # 날짜가 다르면 기존 time_filter 유지
    return {
        "start_date": start_date,
        "end_date": end_date
    }

In [ ]:
from datetime import datetime
from typing import Optional
from pydantic import BaseModel
import instructor
from pydantic import BaseModel, Field, field_validator
from typing import Literal


class TimeFilter(BaseModel):
    start_date: Optional[datetime] = None
    end_date: Optional[datetime] = None

class SearchQuery(BaseModel):
    query: str
    time_filter: TimeFilter

class Label(BaseModel):
    chunk_id: int = Field(description="The unique identifier of the text chunk")
    chain_of_thought: str = Field(
        description="The reasoning process used to evaluate the relevance"
    )
    relevancy: int = Field(
        description="Relevancy score from 0 to 10, where 10 is most relevant",
        ge=0,
        le=10,
    )

class RerankedResults(BaseModel):
    labels: list[Label] = Field(description="List of labeled and ranked chunks")

    @field_validator("labels")
    @classmethod
    def model_validate(cls, v: list[Label]) -> list[Label]:
        return sorted(v, key=lambda x: x.relevancy, reverse=True)

def rerank_results(query: str, chunks: list[dict]) -> RerankedResults:
    # HCX-005 모델을 사용하는 ChatClovaX 인스턴스 생성
    chat = ChatClovaX(
        model="HCX-005",
    )

    # 프롬프트 구성
    system_prompt = """
        You are an expert search result ranker. Your task is to evaluate the relevance of each text chunk to the given query and assign a relevancy score.

        For each chunk:
        1. Analyze its content in relation to the query.
        2. Provide a chain of thought explaining your reasoning.
        3. Assign a relevancy score from 0 to 10, where 10 is most relevant.

        Be objective and consistent in your evaluations.
        """

    # chunk 정보를 텍스트로 변환
    chunk_text = ""
    for chunk in chunks:
        chunk_text += f'<chunk id="{chunk["id"]}">\n{chunk["text"]}\n</chunk>\n'

    user_prompt = f"""
        <query>{query}</query>

        <chunks_to_rank>
        {chunk_text}
        </chunks_to_rank>

        Please provide a JSON array of objects with keys: id, score, reasoning.
        """

    # HCX-005에 메시지 전달
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    response = chat.invoke(messages)
    # response.content에 결과 JSON이 들어있다고 가정
    import json
    labels = json.loads(response.content)
    return RerankedResults(labels=[Label(**label) for label in labels])

In [15]:
get_query_date('2025년 12월 25일')

AttributeError: 'str' object has no attribute 'start_date'

In [ ]:
def get_query_date(question):
    today = datetime(2025, 1, 25)
    days_since_last_friday = (today.weekday() - 4) % 7
    last_friday = today - timedelta(days=days_since_last_friday)
    issue_date = last_friday.strftime("%Y-%m-%d")

    # ChatClovaX 인스턴스 생성
    chat = ChatClovaX(
        model="HCX-005",  # 가장 성능 좋은 모델
    )

    system_prompt = f"""
        You are an AI assistant that extracts date ranges from financial queries.
        The current report date is {issue_date}.
        Your task is to extract the relevant date or date range from the user's query
        and format it in YYYY-MM-DD format.
        If no date is specified, answer with None value.
        Return your answer as a JSON object: {{"time_filter": "YYYY-MM-DD to YYYY-MM-DD"}} or {{"time_filter": null}}
    """

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]
    
    response = chat.invoke(messages)
    import json
    try:
        data = json.loads(response.content)
        time_filter = data.get("time_filter", None)
    except Exception:
        time_filter = None

    parsed_dates = adjust_time_filter_to_week(time_filter)

    if parsed_dates:
        start = parsed_dates['start_date']
        end = parsed_dates['end_date']
    else:
        start = None
        end = None

    if start is None or end is None:
        expr = None
    else:
        expr = f"issue_date >= '{start.strftime('%Y%m%d')}' AND issue_date <= '{end.strftime('%Y%m%d')}'"
    return expr

In [7]:
def convert_to_list(example):
    if isinstance(example["contexts"], list):
        contexts = example["contexts"]
    else:
        try:
            contexts = json.loads(example["contexts"])
        except json.JSONDecodeError as e:
            print(f"JSON Decode Error: {example['contexts']} - {e}")
            contexts = []
    return {"contexts": contexts}

def generate_expr(question: str) -> dict:
    expr = get_query_date(question)
    return {"expr": expr}

def reranking(docs, question, k=15):
    chunks = [{"id": idx, "issue_date": doc.metadata['issue_date'],  "text": doc.page_content} for idx, doc in enumerate(docs)]
    documents_with_metadata = [{"text": doc.page_content, "metadata": doc.metadata} for doc in docs]
    reranked_results = rerank_results(query=question, chunks=chunks)

    chunk_dict = {chunk["id"]: chunk["text"] for chunk in chunks}
    top_k_results = [chunk_dict.get(label.chunk_id, "") for label in reranked_results.labels[:k] if label.chunk_id in chunk_dict]

    reranked_results_with_metadata = []
    for reranked_result in top_k_results:
        page_content = reranked_result

        matching_metadata = None
        for doc in documents_with_metadata:
            if doc["text"] == page_content:
                matching_metadata = doc["metadata"]
                break

        document = Document(
            metadata=matching_metadata,
            page_content=page_content
        )
        reranked_results_with_metadata.append(document)

    context_rerankedNbm25 = reranked_results_with_metadata
    return context_rerankedNbm25

text_prompt = PromptTemplate.from_template(
'''
today is '2025-01-25'. You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
If question has date expressions, context already filtered with the date expression, so ignore about the date and answer without it.
Answer in Korean. Answer in detail.

#Question:
{question}
#Context:
{context}

#Answer:'''
)


In [8]:
question_answer_relevant = GroundednessChecker(
  llm=ChatClovaX(model="HCX-005"), target='question-answer'
).create()

@chain
def kill_table(result):
    if question_answer_relevant.invoke({'question': result['question'], 'answer': result['text']}).score == 'no':
        result['context'] = table_chain.invoke({'question': result['question']})
    else:
        result['context'] = result['text']
    return result

In [9]:
from langchain_naver import ChatClovaX

chat = ChatClovaX(
    model="HCX-005"
)
chat.invoke("hi!")


AIMessage(content='안녕하세요! 저는 CLOVA X입니다.\n\n궁금하신 내용이나 도움이 필요한 사항이 있으시면 말씀해 주세요. 제가 알고 있는 지식과 능력으로 최대한 도움을 드리겠습니다.\n\n즐거운 하루 보내세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 7, 'total_tokens': 50, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'HCX-005', 'system_fingerprint': None, 'id': '80678fa8fdf7416d8c1c76b3ab3cbb65', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--addc7cd2-8f9e-4d3a-9c48-d2fbfe09bebf-0', usage_metadata={'input_tokens': 7, 'output_tokens': 43, 'total_tokens': 50, 'input_token_details': {}, 'output_token_details': {}})

In [10]:
URI = 'http://127.0.0.1:19530'

text_db = Milvus(
    embedding_function=embeddings,
    connection_args = {'uri': URI},
    index_params={'index_type': 'AUTOINDEX', 'metric_type': 'IP'},
    collection_name='text_db'
)

image_db = Milvus(
    embedding_function=embeddings,
    connection_args = {'uri': URI},
    index_params={'index_type': 'AUTOINDEX', 'metric_type': 'IP'},
    collection_name='image_db'
)

raptor_db = Milvus(
    embedding_function=embeddings,
    connection_args = {'uri': URI},
    index_params={'index_type': 'AUTOINDEX', 'metric_type': 'IP'},
    collection_name='raptor_db'
)

table_db = Milvus(
    embedding_function=embeddings,
    connection_args = {'uri': URI},
    index_params={'index_type': 'AUTOINDEX', 'metric_type': 'IP'},
    collection_name='table_db'
)

In [11]:
filepath = 'chunked_jsonl/table_v7.jsonl'

splitted_doc_table = []
with open(filepath, 'r', encoding='utf-8') as file:
    for line in file:
        if line.startswith('\n('):
            continue
        data = json.loads(line)

        doc = Document(
            page_content=data['page_content'],
            metadata=data['metadata']
        )
        splitted_doc_table.append(doc)

In [12]:
bm25_retriever_table = KiwiBM25Retriever.from_documents(
    splitted_doc_table
)
bm25_retriever_table.k = 5

In [13]:
def format_docs(docs):
    # 각 문서의 issue_date와 page_content를 함께 출력하도록 포맷합니다.
    return "\n\n".join(
        f"Issue Date: {doc.metadata.get('issue_date', 'Unknown')}\nContent: {doc.page_content}"
        for doc in docs
    )

In [14]:
get_query_date('연준의 비공식 대변인은?')

AttributeError: 'NoneType' object has no attribute 'start_date'

In [127]:
llm_text = ChatClovaX(model='HCX-005')

answer = []

text_chain = (
    RunnableParallel(
        question=itemgetter('question')
    ).assign(expr = lambda x: get_query_date(x['question'])
    ).assign(context_raw=lambda x: RunnableLambda(
            lambda _: text_db.as_retriever(
                search_kwargs={'expr': x['expr'], 'k': 25}
            ).invoke(x['question'])
        ).invoke({}),
    ).assign(
        formatted_context=lambda x: format_docs(x['context_raw'])
    )
    | RunnableLambda(
        lambda x: {
            "question": x['question'],
            "context": x['formatted_context'],  
        }
    )
    | text_prompt
    | llm_text
    | StrOutputParser()
)

In [ ]:
.assign(
        context=lambda x: reranking(
            list({doc.metadata.get("pk"): doc for doc in (x['context_raw'])}.values()),
            x['question'], 15
        )
    )

In [130]:
text_chain.invoke({'question': '2024년 10월 첫째 주 국내 채권시장 동향'})

'2024년 10월 첫째 주 국내 채권시장은 국제 유가의 급등과 외국인의 매도세 등으로 인해 전반적으로 약세를 보였습니다. 특히 4일에는 국고채 3년물의 최종호가 수익률이 전장 대비 4.4bp 상승해 2.824%에 도달하고, 10년물은 6.4bp 상승하여 2.996%로 마감되었습니다. \n\n그러나 초기에는 미국의 개인소비지출 가격지수의 예상치 하회로 인한 강세의 영향을 받아 긍정적인 시작을 했습니다. 이후 이란의 이스라엘 공습으로 안전자산 선호심리가 강화되면서 미국 국채 금리의 하락에 따라 국내 국고채 금리도 강세를 보였으며, 이는 국고 3년 금리가 장중 2.775%까지 내려가는 결과를 낳았습니다. 그러나 장 막판에 이르러서는 수급 요인으로 인해 제한적인 강세에 그치며 결국 하락 마감했습니다.\n\n기타 금융채 발행액은 전주 대비 크게 감소한 7,900억원을 기록했으며, 회사채 발행 금액은 1조 200억 원이고 만기 금액은 6,110억 원으로 집계되어 총 4,090억 원의 순 발행을 기록했습니다. 은행채 발행액 역시 전주 대비 크게 감소하여 1조 7,400억 원을 기록하였습니다.\n\n단기사채 발행액도 전주 대비 감소하여 169,638억 원을 기록했으며, ELS 발행 건수와 발행 총액 역시 전주 대비 줄어들어 각각 1.76조원을 기록했습니다. 정기예금 발행금액은 전주 대비 감소한 7,301억 원이었으며, 부동산 관련 대출 발행 금액은 증가하여 12,978억 원을 기록했습니다.\n\nCD금리(AAA등급 3개월물 기준)는 전주 대비 약간 하락하여 3.52%로 마감되었고, CP금리(91일, A1등급 기준)는 전주와 비슷한 수준인 3.53%로 마감되었습니다. 이와 함께 회사채 유통 시장에서의 거래도 전주 대비 부진하게 이루어져 전체 유통량이 2조 7,438억 원으로 감소했습니다.\n\n국고 5년물과 10년물의 수익률은 각각 소폭 상승 및 하락으로 마감되었으며, CD 발행은 국내 은행 5곳에서 총 5,'

In [104]:
import json

def parse_search_query_response(response: str, question: str) -> SearchQuery:
    """
    ChatClovaX 응답을 SearchQuery 객체로 파싱
    """
    try:
        # 응답이 JSON 문자열이라고 가정
        data = json.loads(response.content)
        # time_filter가 null이면 빈 dict으로 변환
        if data.get("time_filter") is None:
            data["time_filter"] = {}
        # query 필드가 없으면 원본 question을 사용
        if "query" not in data:
            data["query"] = question
        return SearchQuery(**data)
    except Exception:
        # 파싱 실패 시, 기본값 반환
        return SearchQuery(query=question, time_filter=TimeFilter())


In [102]:
json.loads(response.content)

{'query': '2025년 2월 둘째 주의 국공채 발행액은?',
 'time_filter': {'start_date': '2025-02-08', 'end_date': '2025-02-14'}}

In [114]:
def get_query_date_small(question):
    today = datetime(2025, 1, 25)
    days_since_last_friday = (today.weekday() - 4) % 7
    last_friday = today - timedelta(days=days_since_last_friday)
    issue_date = last_friday.strftime("%Y-%m-%d")

    # ChatClovaX 인스턴스 생성
    chat = ChatClovaX(
        model="HCX-005",
        temperature = 0
    )

    # 프롬프트: 반드시 SearchQuery 포맷(JSON)으로만 답변하게 유도
    system_prompt = f"""
    You are an AI assistant that extracts date ranges from financial queries.
    The current report date is {issue_date}.
    Your task is to extract the relevant date or date range from the user's query
    and format it in YYYY-MM-DD format.
    If no date is specified, answer with None value.
    Return your answer as a JSON object in this format:
    {{
        "query": "<원본 질문>",
        "time_filter": {{"start_date": "YYYY-MM-DD", "end_date": "YYYY-MM-DD"}} or {{"start_date": null, "end_date": null}}
    }}
    답변은 반드시 위 JSON 형태로만 해.
    """

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question},
    ]
    
    response = chat.invoke(messages)
    # ChatClovaX 응답을 SearchQuery로 파싱
    # search_query = parse_search_query_response(response, question)

    # adjust_time_filter_to_week는 기존 함수 그대로 사용
    # parsed_dates = adjust_time_filter_to_week(search_query.time_filter)

    # if parsed_dates:
    #     start = parsed_dates['start_date']
    #     end = parsed_dates['end_date']
    # else:
    #     start = None
    #     end = None

    # if start is None or end is None:
    #     expr = None
    # else:
    #     expr = f"issue_date >= '{start.strftime('%Y%m%d')}' AND issue_date <= '{end.strftime('%Y%m%d')}'"
    return response


In [119]:
get_query_date_small('2025년 1년간 국고채 수익률은?')

AIMessage(content='{\n    "query": "2025년 1년간 국고채 수익률",\n    "time_filter": {"start_date": "2024-01-01", "end_date": "2025-12-31"}\n}', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 158, 'total_tokens': 207, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'HCX-005', 'system_fingerprint': None, 'id': '57abe0622a5941f3b9f46f699a1f021c', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--e3158487-45ab-4a20-85ac-0603ce83810e-0', usage_metadata={'input_tokens': 158, 'output_tokens': 49, 'total_tokens': 207, 'input_token_details': {}, 'output_token_details': {}})

In [107]:
def get_query_date(question):
    today = datetime(2025, 1, 25)
    days_since_last_friday = (today.weekday() - 4) % 7
    last_friday = today - timedelta(days=days_since_last_friday)
    issue_date = last_friday.strftime("%Y-%m-%d")

    # ChatClovaX 인스턴스 생성
    chat = ChatClovaX(
        model="HCX-005",
        temperature = 0
    )

    # 프롬프트: 반드시 SearchQuery 포맷(JSON)으로만 답변하게 유도
    system_prompt = f"""
    You are an AI assistant that extracts date ranges from financial queries.
    The current report date is {issue_date}.
    Your task is to extract the relevant date or date range from the user's query
    and format it in YYYY-MM-DD format.
    If no date is specified, answer with None value.
    Return your answer as a JSON object in this format:
    {{
        "query": "<원본 질문>",
        "time_filter": {{"start_date": "YYYY-MM-DD", "end_date": "YYYY-MM-DD"}} or {{"start_date": null, "end_date": null}}
    }}
    답변은 반드시 위 JSON 형태로만 해.
    """

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question},
    ]
    
    response = chat.invoke(messages)
    # ChatClovaX 응답을 SearchQuery로 파싱
    search_query = parse_search_query_response(response, question)

    # adjust_time_filter_to_week는 기존 함수 그대로 사용
    parsed_dates = adjust_time_filter_to_week(search_query.time_filter)

    if parsed_dates:
        start = parsed_dates['start_date']
        end = parsed_dates['end_date']
    else:
        start = None
        end = None

    if start is None or end is None:
        expr = None
    else:
        expr = f"issue_date >= '{start.strftime('%Y%m%d')}' AND issue_date <= '{end.strftime('%Y%m%d')}'"
    return expr


In [123]:
get_query_date('연준의 비공식 대변인은?')

In [112]:
get_query_date('2025년 1년간 채권 발행액')

"issue_date >= '20240101' AND issue_date <= '20251231'"

In [74]:
expr

In [86]:
def elsethings(response, question):
    search_query = parse_search_query_response(response, question)
    parsed_dates = adjust_time_filter_to_week(search_query.time_filter)

    if parsed_dates:
        start = parsed_dates['start_date']
        end = parsed_dates['end_date']
    else:
        start = None
        end = None

    if start is None or end is None:
        expr = None
    else:
        expr = f"issue_date >= '{start.strftime('%Y%m%d')}' AND issue_date <= '{end.strftime('%Y%m%d')}'"
    return expr

In [105]:
print(elsethings(response, '2025년 2월 둘째 주의 국공채 발행액은?'))

issue_date >= '20250208' AND issue_date <= '20250214'


In [ ]:
res

In [100]:
type(response.content)

str

In [95]:
response

AIMessage(content='{\n    "query": "2025년 2월 둘째 주의 국공채 발행액은?",\n    "time_filter": {"start_date": "2025-02-08", "end_date": "2025-02-14"}\n}', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 162, 'total_tokens': 215, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'HCX-005', 'system_fingerprint': None, 'id': '5a13f6ebd8c340b6a734e11da05ab032', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--630f0040-31f3-4b23-afe6-62e58c6bac84-0', usage_metadata={'input_tokens': 162, 'output_tokens': 53, 'total_tokens': 215, 'input_token_details': {}, 'output_token_details': {}})

In [94]:
parse_search_query_response(response, '2025년 2월 둘째 주의 국공채 발행액은?')

SearchQuery(query='2025년 2월 둘째 주의 국공채 발행액은?', time_filter=TimeFilter(start_date=None, end_date=None))

In [93]:
response

NameError: name 'parsed_dates' is not defined

In [ ]:
from datetime import datetime
from typing import Dict, Any
from pydantic import BaseModel, Field
import json

# 평가 결과 모델
class GroundnessQuestionScore(BaseModel):
    score: str = Field(
        description="질문과 답변이 명확히 관련 있으면 'yes', 아니면 'no'"
    )

class GroundnessAnswerRetrievalScore(BaseModel):
    score: str = Field(
        description="검색된 문서와 답변이 명확히 관련 있으면 'yes', 아니면 'no'"
    )

class GroundnessQuestionRetrievalScore(BaseModel):
    score: str = Field(
        description="검색된 문서와 질문이 명확히 관련 있으면 'yes', 아니면 'no'"
    )

In [ ]:
class GroundednessCheckerClovaX:
    """
    하이퍼클로바 X 기반의 정확성 평가기 (한글 프롬프트 + yes/no 단일값 예시)
    """

    def __init__(self, llm, target="retrieval-answer"):
        self.target = target
        self.llm = llm

    def _get_prompt_and_inputs(self, **kwargs):
        if self.target == "retrieval-answer":
            template = """
당신은 검색된 문서가 사용자 답변과 얼마나 관련이 있는지 평가하는 평가자입니다.
아래는 검색된 문서입니다:
{context}
아래는 답변입니다:
{answer}
문서에 답변과 관련된 키워드나 의미가 명확하게 포함되어 있다면 '관련 있음'으로 평가하세요.
문서에 답변과 관련된 정보가 전혀 없다면 '관련 없음'으로 평가하세요.
엄격하게 평가하세요. 명확히 관련 있을 때만 'yes'로 답하세요.

예시:
문서: "애플은 미국의 대표적인 IT 기업입니다."
답변: "애플은 미국 회사입니다."
점수: yes

문서: "삼성전자는 반도체 시장에서 큰 비중을 차지합니다."
답변: "애플은 미국 회사입니다."
점수: no

반드시 'yes' 또는 'no'만 단독으로 답변하세요.
"""
            input_vars = ["context", "answer"]

        elif self.target == "question-answer":
            template = """
당신은 주어진 답변이 질문에 적절하게 답하고 있는지 평가하는 평가자입니다.
아래는 질문입니다:
{question}
아래는 답변입니다:
{answer}
답변이 질문에 직접적으로 답하고 있고 관련 정보가 포함되어 있다면 '관련 있음'으로 평가하세요.
답변이 질문에 답하지 않거나 무관하다면 '관련 없음'으로 평가하세요.
엄격하게 평가하세요. 명확히 관련 있을 때만 'yes'로 답하세요.

예시:
질문: "프랑스의 수도는 어디인가요?"
답변: "프랑스의 수도는 파리입니다."
점수: yes

질문: "프랑스의 수도는 어디인가요?"
답변: "프랑스는 유럽의 국가입니다."
점수: no

질문: "프랑스의 수도는 어디인가요?"
답변: "독일의 수도는 베를린입니다."
점수: no

반드시 'yes' 또는 'no'만 단독으로 답변하세요.
"""
            input_vars = ["question", "answer"]

        elif self.target == "question-retrieval":
            template = """
당신은 검색된 문서가 주어진 질문에 답하는 데 도움이 되는지 평가하는 평가자입니다.
아래는 질문입니다:
{question}
아래는 검색된 문서입니다:
{context}
문서에 질문에 답하는 데 도움이 되는 정보가 명확하게 포함되어 있다면 '관련 있음'으로 평가하세요.
문서에 질문에 도움이 되는 정보가 전혀 없다면 '관련 없음'으로 평가하세요.
엄격하게 평가하세요. 명확히 관련 있을 때만 'yes'로 답하세요.

예시:
질문: "프랑스의 수도는 어디인가요?"
문서: "파리는 프랑스의 수도입니다."
점수: yes

질문: "프랑스의 수도는 어디인가요?"
문서: "프랑스는 유럽의 국가입니다."
점수: no

질문: "프랑스의 수도는 어디인가요?"
문서: "베를린은 독일의 수도입니다."
점수: no

반드시 'yes' 또는 'no'만 단독으로 답변하세요.
"""
            input_vars = ["question", "context"]

        else:
            raise ValueError(f"Invalid target: {self.target}")

        prompt = template.format(**{k: kwargs[k] for k in input_vars})
        return prompt, input_vars

    def __call__(self, **kwargs):
        prompt, input_vars = self._get_prompt_and_inputs(**kwargs)
        messages = [
            {"role": "system", "content": prompt}
        ]
        response = self.llm.invoke(messages)
        # 응답에서 yes/no만 추출
        # answer = response.strip().replace('"', '').replace("'", "").lower()
        # if answer not in ["yes", "no"]:
        #     # 혹시라도 모델이 다른 텍스트를 내보내면 yes/no가 포함된 부분만 추출
        #     import re
        #     match = re.search(r'\b(yes|no)\b', answer)
        #     if match:
        #         answer = match.group(1)
        #     else:
        #         raise ValueError(f"Invalid response: {response}")

        # # Pydantic 모델 매핑
        # if self.target == "retrieval-answer":
        #     return GroundnessAnswerRetrievalScore(score=answer)
        # elif self.target == "question-answer":
        #     return GroundnessQuestionScore(score=answer)
        # elif self.target == "question-retrieval":
        #     return GroundnessQuestionRetrievalScore(score=answer)
        return response

In [201]:
gc(question='미국의 수도는 어디인가요?', answer='서울은 대한민국의 수도입니다')

[{'role': 'system',
  'content': '\n당신은 주어진 답변이 질문에 적절하게 답하고 있는지 평가하는 평가자입니다.\n아래는 질문입니다:\n미국의 수도는 어디인가요?\n아래는 답변입니다:\n서울은 대한민국의 수도입니다\n답변이 질문에 직접적으로 답하고 있고 관련 정보가 포함되어 있다면 \'관련 있음\'으로 평가하세요.\n답변이 질문에 답하지 않거나 무관하다면 \'관련 없음\'으로 평가하세요.\n엄격하게 평가하세요. 명확히 관련 있을 때만 \'yes\'로 답하세요.\n\n예시:\n질문: "프랑스의 수도는 어디인가요?"\n답변: "프랑스의 수도는 파리입니다."\n점수: yes\n\n질문: "프랑스의 수도는 어디인가요?"\n답변: "프랑스는 유럽의 국가입니다."\n점수: no\n\n질문: "프랑스의 수도는 어디인가요?"\n답변: "독일의 수도는 베를린입니다."\n점수: no\n\n반드시 \'yes\' 또는 \'no\'만 단독으로 답변하세요.\n'}]

In [197]:
llm = ChatClovaX(model='HCX-003')

In [198]:
gc = GroundednessCheckerClovaX(llm, target='question-answer')

In [192]:
gc(question='미국의 수도는 어디인가요?', answer='서울은 대한민국의 수도입니다')

AttributeError: 'list' object has no attribute 'strip'

In [246]:
class GroundednessCheckerClovaX:
    """
    HyperCLOVA X 기반의 정확성 평가기 (OpenAI 버전과 동일한 구조)
    """

    def __init__(self, llm, target="retrieval-answer"):
        self.llm = llm
        self.target = target

    def create(self):
        # 프롬프트 선택
        if self.target == "retrieval-answer":
            template = """당신은 검색된 문서가 사용자 답변과 얼마나 관련이 있는지 평가하는 평가자입니다.
아래는 검색된 문서입니다:
{context}
아래는 답변입니다:
{answer}
문서에 답변과 관련된 키워드나 의미가 명확하게 포함되어 있다면 '관련 있음'으로 평가하세요.
문서에 답변과 관련된 정보가 전혀 없다면 '관련 없음'으로 평가하세요.
엄격하게 평가하세요. 명확히 관련 있을 때만 'yes'로 답하세요.

예시:
문서: "애플은 미국의 대표적인 IT 기업입니다."
답변: "애플은 미국 회사입니다."
점수: yes

문서: "삼성전자는 반도체 시장에서 큰 비중을 차지합니다."
답변: "애플은 미국 회사입니다."
점수: no

반드시 'yes' 또는 'no'만 단독으로 답변하세요.
"""
            input_vars = ["context", "answer"]

        elif self.target == "question-answer":
            template = """당신은 주어진 답변이 질문에 적절하게 답하고 있는지 평가하는 평가자입니다.
아래는 질문입니다:
{question}
아래는 답변입니다:
{answer}
답변이 질문에 직접적으로 답하고 있고 관련 정보가 포함되어 있다면 '관련 있음'으로 평가하세요.
답변이 질문에 답하지 않거나 무관하다면 '관련 없음'으로 평가하세요.
엄격하게 평가하세요. 명확히 관련 있을 때만 'yes'로 답하세요.

예시:
질문: "프랑스의 수도는 어디인가요?"
답변: "프랑스의 수도는 파리입니다."
점수: yes

질문: "프랑스의 수도는 어디인가요?"
답변: "프랑스는 유럽의 국가입니다."
점수: no

질문: "프랑스의 수도는 어디인가요?"
답변: "독일의 수도는 베를린입니다."
점수: no

반드시 'yes' 또는 'no'만 단독으로 답변하세요.
"""
            input_vars = ["question", "answer"]

        elif self.target == "question-retrieval":
            template = """당신은 검색된 문서가 주어진 질문에 답하는 데 도움이 되는지 평가하는 평가자입니다.
아래는 질문입니다:
{question}
아래는 검색된 문서입니다:
{context}
문서에 질문에 답하는 데 도움이 되는 정보가 명확하게 포함되어 있다면 '관련 있음'으로 평가하세요.
문서에 질문에 도움이 되는 정보가 전혀 없다면 '관련 없음'으로 평가하세요.
엄격하게 평가하세요. 명확히 관련 있을 때만 'yes'로 답하세요.

예시:
질문: "프랑스의 수도는 어디인가요?"
문서: "파리는 프랑스의 수도입니다."
점수: yes

질문: "프랑스의 수도는 어디인가요?"
문서: "프랑스는 유럽의 국가입니다."
점수: no

질문: "프랑스의 수도는 어디인가요?"
문서: "베를린은 독일의 수도입니다."
점수: no

반드시 'yes' 또는 'no'만 단독으로 답변하세요.
"""
            input_vars = ["question", "context"]

        else:
            raise ValueError(f"Invalid target: {self.target}")

        def chain_func(**kwargs):
            prompt = template.format(**{k: kwargs[k] for k in input_vars})
            messages = [
                {"role": "system", "content": prompt}
            ]
            print(messages)
            response = self.llm.invoke(messages)
            print(response)
            # 응답에서 yes/no만 추출
            answer = response.content.strip().replace('"', '').replace("'", "").lower()
            if answer not in ["yes", "no"]:
                import re
                match = re.search(r'\b(yes|no)\b', answer)
                if match:
                    answer = match.group(1)
                else:
                    raise ValueError(f"Invalid response: {response}")

            # Pydantic 모델 매핑
            if self.target == "retrieval-answer":
                return GroundnessAnswerRetrievalScore(score=answer)
            elif self.target == "question-answer":
                return GroundnessQuestionScore(score=answer)
            elif self.target == "question-retrieval":
                return GroundnessQuestionRetrievalScore(score=answer)

        return chain_func

In [247]:
gc = GroundednessCheckerClovaX(llm=llm, target='question-answer').create()

In [241]:
from langchain_naver import ChatClovaX

llm = ChatClovaX(model='HCX-005', temperature=0)

In [236]:
llm.invoke('hi')

AIMessage(content='안녕하세요! 저는 CLOVA X입니다.\n\n궁금하신 부분이나 도움이 필요하신 일이 있으시다면 말씀해 주세요. 제가 알고 있는 지식과 능력으로 최대한 도움을 드리겠습니다.\n\n좋은 하루 보내세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 6, 'total_tokens': 49, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'HCX-005', 'system_fingerprint': None, 'id': '3d88fca92dd84bc8998f7e1a139dad03', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--50af2c54-54bf-44fc-8581-2d9389ad4a6d-0', usage_metadata={'input_tokens': 6, 'output_tokens': 43, 'total_tokens': 49, 'input_token_details': {}, 'output_token_details': {}})

In [237]:
llm.invoke([{'role': 'system', 'content': '당신은 주어진 답변이 질문에 적절하게 답하고 있는지 평가하는 평가자입니다.\n아래는 질문입니다:\n미국의 수도는 어디인가요?\n아래는 답변입니다:\n서울은 대한민국의 수도입니다\n답변이 질문에 직접적으로 답하고 있고 관련 정보가 포함되어 있다면 \'관련 있음\'으로 평가하세요.\n답변이 질문에 답하지 않거나 무관하다면 \'관련 없음\'으로 평가하세요.\n엄격하게 평가하세요. 명확히 관련 있을 때만 \'yes\'로 답하세요.\n\n예시:\n질문: "프랑스의 수도는 어디인가요?"\n답변: "프랑스의 수도는 파리입니다."\n점수: yes\n\n질문: "프랑스의 수도는 어디인가요?"\n답변: "프랑스는 유럽의 국가입니다."\n점수: no\n\n질문: "프랑스의 수도는 어디인가요?"\n답변: "독일의 수도는 베를린입니다."\n점수: no\n\n반드시 \'yes\' 또는 \'no\'만 단독으로 답변하세요.\n'}])

AIMessage(content='no', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 216, 'total_tokens': 218, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'HCX-005', 'system_fingerprint': None, 'id': 'e3629b1c3a1d4a459e649733fd4ae4ac', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--16a73ea9-8fb1-41d4-a90d-36a4a9211422-0', usage_metadata={'input_tokens': 216, 'output_tokens': 2, 'total_tokens': 218, 'input_token_details': {}, 'output_token_details': {}})

In [228]:
print(gc)

<function GroundednessCheckerClovaX.create.<locals>.chain_func at 0x00000138080C6D40>


In [248]:
messages = gc(question='미국의 수도는 어디인가요?', answer='서울은 대한민국의 수도입니다')

[{'role': 'system', 'content': '당신은 주어진 답변이 질문에 적절하게 답하고 있는지 평가하는 평가자입니다.\n아래는 질문입니다:\n미국의 수도는 어디인가요?\n아래는 답변입니다:\n서울은 대한민국의 수도입니다\n답변이 질문에 직접적으로 답하고 있고 관련 정보가 포함되어 있다면 \'관련 있음\'으로 평가하세요.\n답변이 질문에 답하지 않거나 무관하다면 \'관련 없음\'으로 평가하세요.\n엄격하게 평가하세요. 명확히 관련 있을 때만 \'yes\'로 답하세요.\n\n예시:\n질문: "프랑스의 수도는 어디인가요?"\n답변: "프랑스의 수도는 파리입니다."\n점수: yes\n\n질문: "프랑스의 수도는 어디인가요?"\n답변: "프랑스는 유럽의 국가입니다."\n점수: no\n\n질문: "프랑스의 수도는 어디인가요?"\n답변: "독일의 수도는 베를린입니다."\n점수: no\n\n반드시 \'yes\' 또는 \'no\'만 단독으로 답변하세요.\n'}]
content='no' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 216, 'total_tokens': 218, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'HCX-005', 'system_fingerprint': None, 'id': '29cc6b6cf6304347a0ca6e8d101b0181', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--f0569120-cad4-458f-be1b-b863ccec30e6-0' usage_metadata={'input_tokens': 216, 'output_tokens': 2, 'total_tokens': 218, 'input_

In [249]:
messages

GroundnessQuestionScore(score='no')

In [ ]:
class GroundednessCheckerClovaX:
    """
    HyperCLOVA X 기반의 정확성 평가기 (OpenAI 버전과 동일한 구조)
    """

    def __init__(self, llm, target="retrieval-answer"):
        self.llm = llm
        self.target = target


    def create(self):
        # 프롬프트 선택
        if self.target == "retrieval-answer":
            template = """
You are a grader assessing relevance of a retrieved document to a user question.
Here is the retrieved document: {context}
Here is the answer: {answer}
If the document contains keyword(s) or semantic meaning related to the user answer, grade it as relevant.
Return your answer as a JSON object: {{"score": "yes"}} or {{"score": "no"}}.
답변은 반드시 위 JSON 형태로만 해.
"""
            input_vars = ["context", "answer"]
        elif self.target == "question-answer":
            template = """
You are a grader assessing whether an answer appropriately addresses the given question.
Here is the question: {question}
Here is the answer: {answer}
If the answer directly addresses the question and provides relevant information, grade it as relevant.
Consider both semantic meaning and factual accuracy in your assessment.
Return your answer as a JSON object: {{"score": "yes"}} or {{"score": "no"}}.
답변은 반드시 위 JSON 형태로만 해.
"""
            input_vars = ["question", "answer"]
        elif self.target == "question-retrieval":
            template = """
You are a grader assessing whether a retrieved document is relevant to the given question.
Here is the question: {question}
Here is the retrieved document: {context}
If the document contains information that could help answer the question, grade it as relevant.
Consider both semantic meaning and potential usefulness for answering the question.
Return your answer as a JSON object: {{"score": "yes"}} or {{"score": "no"}}.
답변은 반드시 위 JSON 형태로만 해.
"""
            input_vars = ["question", "context"]
        else:
            raise ValueError(f"Invalid target: {self.target}")

        def chain_func(**kwargs):
            prompt = template.format(**{k: kwargs[k] for k in input_vars})
            messages = [
                {"role": "system", "content": prompt}
            ]
            response = self.llm.invoke(messages)
            # 응답에서 yes/no만 추출
            answer = response.content.strip().replace('"', '').replace("'", "").lower()
            if answer not in ["yes", "no"]:
                import re
                match = re.search(r'\b(yes|no)\b', answer)
                if match:
                    answer = match.group(1)
                else:
                    raise ValueError(f"Invalid response: {response}")

            # Pydantic 모델 매핑
            if self.target == "retrieval-answer":
                return GroundnessAnswerRetrievalScore(score=answer)
            elif self.target == "question-answer":
                return GroundnessQuestionScore(score=answer)
            elif self.target == "question-retrieval":
                return GroundnessQuestionRetrievalScore(score=answer)

        return chain_func


In [251]:
gc = GroundednessCheckerClovaX(llm=llm, target='question-answer').create()

In [254]:
gc(question='미국의 수도는 어디인가요?', answer='미국의 수도는 워싱턴입니다')

[{'role': 'system', 'content': '\nYou are a grader assessing whether an answer appropriately addresses the given question.\nHere is the question: 미국의 수도는 어디인가요?\nHere is the answer: 미국의 수도는 워싱턴입니다\nIf the answer directly addresses the question and provides relevant information, grade it as relevant.\nConsider both semantic meaning and factual accuracy in your assessment.\nReturn your answer as a JSON object: {"score": "yes"} or {"score": "no"}.\n답변은 반드시 위 JSON 형태로만 해.\n'}]
content='{"score": "yes"}' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 99, 'total_tokens': 106, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'HCX-005', 'system_fingerprint': None, 'id': '1e79ee06ff6447fcb7493d3925c099e1', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--b01dd22d-d4fc-4871-b684-a8e35a8f443a-0' usage_metadata={'input_tokens': 99, 'output_tokens': 7, 'total_tokens': 106, 'inpu

GroundnessQuestionScore(score='yes')